In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import time
from tqdm import tqdm
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\twentystones\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 포도사전

In [ ]:
#데이터 로드
train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/FoDoDic_ew_진짜_최초라벨링.csv')
print('훈련용 리뷰 개수 :',len(train_data))

In [27]:
#데이터 로드
train_data = pd.read_csv('C:/Users/twentystones/Desktop/1프로젝트/감성사전/포도사전/FoDoDic_ew_진짜_최초라벨링.csv')
print('훈련용 리뷰 개수 :',len(train_data))

훈련용 리뷰 개수 : 101324


In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, = train_test_split(train_data, test_size=0.25, random_state=42)

train_data = X_train
test_data = X_test

#중복 데이터 수
train_data['best_comment'].nunique(), train_data['label'].nunique()

# 중복 제거
train_data.drop_duplicates(subset=['best_comment'], inplace=True) 

# 트레인셋의 라벨 수
print(train_data.groupby('label').size().reset_index(name = 'count'))

   label  count
0      0  20698
1      1  55197


In [29]:
#결측치있다면 삭제
if train_data.isnull().values.any() ==True:
    train_data = train_data.dropna(how = 'any')
    print('결측치 제거 후 갯수')
    len(train_data)

test_data['best_comment'].nunique(), test_data['label'].nunique()
test_data.drop_duplicates(subset=['best_comment'], inplace=True) 
if test_data.isnull().values.any() ==True:
    test_data = test_data.dropna(how = 'any')
    print('결측치 제거 후 갯수')
    len(test_data)
test_data

,number,best_comment,label
29005,96635,I'm so happy Risa didn't completely shit on th...,1
56495,212281,sometimes I forget he's supposed to be the sup...,1
22998,78146,His shorts..😂😂😂😂😂 THIS MAN CAN PULL OFF BOOTY ...,1
38432,135284,Ah I love this sooo much!!! I could read this ...,1
32094,110176,"I love jesus face like "" am sorry bro"" 😂😂😂",1
...,...,...,...
69430,261354,REALIZE YOU SCREWED UP IMMANUEL-,0
9582,29942,"I love Jay, but let's be honest he lost his ch...",1
58410,220030,Aw man I really loved this story it’s sad that...,0
81797,304977,TOPawww omg she’s so lonely 😭 i hope she gets ...,1


In [30]:
X_train = []
for sentence in train_data['best_comment']:
    temp_X =  word_tokenize(sentence) # 형태소분리
    X_train.append(temp_X)

X_test = []
for sentence in test_data['best_comment']:
  temp_X =  word_tokenize(sentence) # 토큰화
  X_test.append(temp_X)

In [31]:
#정수인코딩
tokenizer = Tokenizer()
tokenizer.fit_on_texts(str(X_train))

tokenizer.fit_on_texts(str(X_test))

print(tokenizer.word_index)

{"'": 1, 'e': 2, 't': 3, 'o': 4, 'a': 5, 'i': 6, 's': 7, 'n': 8, 'h': 9, 'r': 10, 'l': 11, 'd': 12, 'u': 13, 'm': 14, 'y': 15, 'c': 16, 'g': 17, 'w': 18, 'p': 19, 'f': 20, 'b': 21, 'k': 22, 'v': 23, 'j': 24, '’': 25, '😂': 26, 'x': 27, 'z': 28, '😭': 29, '0': 30, 'q': 31, '️': 32, '❤': 33, '“': 34, '2': 35, '”': 36, '1': 37, '😍': 38, '3': 39, '✨': 40, '😅': 41, '🤣': 42, '5': 43, '🥺': 44, '💕': 45, '4': 46, '⣿': 47, '9': 48, '👏': 49, '6': 50, '🥰': 51, '7': 52, '8': 53, '⠀': 54, '💖': 55, '♡': 56, '♥': 57, '😊': 58, '😢': 59, '🏻': 60, '😆': 61, '😁': 62, '💜': 63, '🤔': 64, '👌': 65, '😏': 66, '👀': 67, '…': 68, '—': 69, '‘': 70, '♀': 71, '😱': 72, '°': 73, '🏼': 74, '💀': 75, '😔': 76, '👍': 77, '😤': 78, '😩': 79, '😳': 80, '͡': 81, '💛': 82, '😌': 83, '💙': 84, '☺': 85, '😄': 86, '😘': 87, '🎉': 88, '🏽': 89, '💚': 90, '💗': 91, '💔': 92, '👁': 93, '😫': 94, '🔥': 95, '•': 96, '😉': 97, '🌈': 98, '🙏': 99, '🤷': 100, '🙌': 101, '▒': 102, '░': 103, '🤩': 104, '😋': 105, '🏾': 106, '🤦': 107, '🙄': 108, '😎': 109, '💞': 110, '🧡': 11

In [32]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value
       
    
vocab_size = total_cnt - rare_cnt + 1

tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
tokenizer.fit_on_texts(X_test)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

In [33]:
# 빈도수 적은 단어들 삭제해서 빈 샘플들이 생겼음. 이를 다시 제거
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
print(len(X_train))
print(len(y_train))

75777
75777


C:\Users\twentystones\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [34]:
#지정한 길이 이상길이 샘플 삭제
def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if(len(s) <= max_len):
        cnt = cnt + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

max_len = 80
below_threshold_len(max_len, X_train)


X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

전체 샘플 중 길이가 80 이하인 샘플의 비율: 95.9222455362445


In [43]:
from tensorflow.keras.layers import Embedding, Dense, LSTM, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.datasets import imdb

In [36]:
model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='acc', mode='max', verbose=1, save_best_only=True)


model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)

Epoch 1/15
1011/1011 [==============================] - 97s 95ms/step - loss: 0.2990 - acc: 0.8839 - val_loss: 0.2314 - val_acc: 0.9190

Epoch 00001: acc improved from -inf to 0.88390, saving model to best_model.h5
Epoch 2/15
1011/1011 [==============================] - 94s 93ms/step - loss: 0.1966 - acc: 0.9317 - val_loss: 0.2152 - val_acc: 0.9235

Epoch 00002: acc improved from 0.88390 to 0.93166, saving model to best_model.h5
Epoch 3/15
1011/1011 [==============================] - 98s 97ms/step - loss: 0.1793 - acc: 0.9363 - val_loss: 0.1843 - val_acc: 0.9333

Epoch 00003: acc improved from 0.93166 to 0.93626, saving model to best_model.h5
Epoch 4/15
1011/1011 [==============================] - 111s 109ms/step - loss: 0.1685 - acc: 0.9387 - val_loss: 0.1825 - val_acc: 0.9341

Epoch 00004: acc improved from 0.93626 to 0.93872, saving model to best_model.h5
Epoch 5/15
1011/1011 [==============================] - 123s 121ms/step - loss: 0.1598 - acc: 0.9422 - val_loss: 0.1744 - val_acc

In [37]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

792/792 [==============================] - 22s 28ms/step - loss: 0.1771 - acc: 0.9319

 테스트 정확도: 0.9319


In [38]:
#웹툰댓글파일 불러오기. 한글 37만여 댓글중 라벨링한 데이터 재외한 27만여 댓글들 분석
temp =  pd.read_csv('C:/Users/twentystones/Desktop/1프로젝트/감성사전/포도사전/FoDoDic_ew_진짜_최초라벨링.csv')
webtoon_comment =  pd.read_csv('C:/Users/twentystones/Desktop/1프로젝트/데이터 모음집/ew_2_comment.csv').drop(temp['number'].values.tolist(), axis=0)

#점수매기기
pred_label=[]
for sentence in tqdm(webtoon_comment['best_comment']):
    try:
        pred_label.append(sentiment_predict(sentence)) 
    except :
        pred_label.append(2)

df = pd.DataFrame({'number' :webtoon_comment.index,
                   'LabelfromNaverMovieReview' : pred_label,
                   'comment' : webtoon_comment['best_comment']} )
df

100%|████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<?, ?it/s]


,number,LabelfromNaverMovieReview,comment
0,0,2,TOPthey be looking like a whole POWER couple w...
1,1,2,TOPThe teasing of this back story with Clove’s...
2,2,2,TOPThe Queen: Ultimate cockblock
4,4,2,Why is no one talking about how Kharis' mother...
6,6,2,only kyro would be flirty and trying to kiss c...
...,...,...,...
132,132,2,Hey guys! the new ep should be up soon!
133,133,2,DRAGON UP 👏🏽👏🏽
135,135,2,TOPI hope she’s able to shift and safe them al...
139,139,2,"""I knew I shouldn't have gotten out of bed"" du..."


# IMDB로 라벨링

In [44]:
# 데이터셋 준비
word_to_index = imdb.get_word_index()
index_to_word={}
for key, value in word_to_index.items():
    index_to_word[value+3] = key
    
vocab_size = 20000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocab_size)
X_train = np.append(X_train, X_test)
y_train = np.append(y_train, y_test)
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [45]:
# GRU 모델 훈련
model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(GRU(128))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('GRU_model.h5', monitor='acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)

loaded_model = load_model('GRU_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

Epoch 1/5
667/667 [==============================] - 476s 711ms/step - loss: 0.4200 - acc: 0.8097 - val_loss: 0.3094 - val_acc: 0.8722

Epoch 00001: acc improved from -inf to 0.80972, saving model to GRU_model.h5
Epoch 2/5
667/667 [==============================] - 528s 791ms/step - loss: 0.2583 - acc: 0.9013 - val_loss: 0.2447 - val_acc: 0.9008

Epoch 00002: acc improved from 0.80972 to 0.90132, saving model to GRU_model.h5
Epoch 3/5
667/667 [==============================] - 507s 760ms/step - loss: 0.1802 - acc: 0.9331 - val_loss: 0.2632 - val_acc: 0.8978

Epoch 00003: acc improved from 0.90132 to 0.93308, saving model to GRU_model.h5
Epoch 4/5
667/667 [==============================] - 424s 635ms/step - loss: 0.1406 - acc: 0.9477 - val_loss: 0.2173 - val_acc: 0.9156

Epoch 00004: acc improved from 0.93308 to 0.94765, saving model to GRU_model.h5
Epoch 5/5
667/667 [==============================] - 401s 601ms/step - loss: 0.1148 - acc: 0.9596 - val_loss: 0.2273 - val_acc: 0.9147

Epo

In [49]:
# 영어 웹툰 댓글에 적용
test_data = pd.read_csv('C:/Users/twentystones/Desktop/1프로젝트/데이터 모음집/ew_2_comment.csv')

#정규화
test_data['adj_comment'] = test_data['best_comment'].str.replace("[^0-9a-zA-Z ]","").str.lower().str.replace('^ +', "")

#댓글 앞에 TOP자가 붙은것 제거
for i in range(len(test_data['adj_comment'])):
  if test_data['adj_comment'][i][:3] == 'top':
    test_data['adj_comment'][i] = test_data['adj_comment'][i][3:]
    
#점수책정    
def sentiment_predict(new_sentence):
  encoded = []
  for word in new_sentence.split():
      try :
          if word_to_index[word] <= 20000:
              encoded.append(word_to_index[word]+3)
          else:
              encoded.append(2)
      except KeyError:
          encoded.append(2)
  pad_new = pad_sequences([encoded], maxlen = max_len)
  score = float(loaded_model.predict(pad_new))
  return score

score_list = []
for i in tqdm(test_data['adj_comment']):
  try:
    score_list.append(sentiment_predict(i))
  except:   #오류시 99숫자 넣기
    score_list.append(99)
result_score = pd.DataFrame(score_list, columns = ['score_imdb'])
result_score

<ipython-input-49-e9e037e20b63>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['adj_comment'] = test_data['best_comment'].str.replace("[^0-9a-zA-Z ]","").str.lower().str.replace('^ +', "")


5 s


,score_imdb
0,0.407978
1,0.474016
2,0.783215
3,0.367585
4,0.280987
...,...
95,0.786477
96,0.231399
97,0.972024
98,0.516193
